### 1. Selecting an Embedding Model

In [10]:
%pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [26]:
user_question = "I just discovered the course. Can I still join it?"
question_embedding = embedding_model.encode(user_question)
print(f"Dimensions: {question_embedding.shape[0]}")

Dimensions: 768


The first dimension has the following value:

In [27]:
question_embedding[0]

np.float32(0.07822265)

The embedding is normalized.

In [30]:
import numpy as np

np.linalg.norm(question_embedding)

np.float32(0.99999994)

### 2. Creating Document Embeddings

In [ ]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-evaluation/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'

docs_response = requests.get(docs_url)
documents = docs_response.json()

In [20]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [21]:
course_name = 'machine-learning-zoomcamp'
filtered_docs = [doc for doc in documents if doc['course'] == course_name]
len(filtered_docs)

375

Computing the matrix of embeddings `X`.

In [ ]:
embeddings = []
for doc in filtered_docs:
    qa_text = f'{doc['question']} {doc['text']}'
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

X = np.array(embeddings)

In [25]:
X.shape

(375, 768)

### 3. Building Search Engine

Question vector is normalized, so dot product between matrix and this vector gives us cosine similarity scores.

In [33]:
scores = X.dot(question_embedding)
max(scores)

np.float32(0.6506573)

Now we have everything we need to build our own vector search engine class!

In [ ]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results] # Sort in descending and get doc indices
        return [self.documents[i] for i in idx]

In [35]:
search_engine = VectorSearchEngine(documents=filtered_docs, embeddings=X)
search_engine.search(question_embedding, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### 4. Evaluation Based On Hit-Rate

In [37]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-evaluation/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [38]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [39]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

For each ground truth document, a search is performed and compared with each returned result.

In [43]:
relevance_total = []
for q in ground_truth:
    doc_id = q['document']
    results = search_engine.search(
        v_query=embedding_model.encode(q['question']), 
        num_results=5
    )
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

The hit rate measures how many queries actually hit the actual document, including it in the top 5 most relevant ones.

In [44]:
hit_rate(relevance_total)

0.9398907103825137